In [2]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('sqlite:///dados/banking.db')

juros = pd.read_csv('dados/Taxa de juros de modalidades diárias e mensais.csv', sep=',', encoding='latin1', on_bad_lines='warn')
juros = juros.rename(columns={'cnpj8': 'cnpj', 'FimPeriodo': 'data'})
juros['data'] = pd.to_datetime(juros['data'])
juros = juros.drop(columns=['InicioPeriodo', 'TaxaJurosAoMes'])
juros_consignado_privado = juros[juros['codigoModalidade'] == 219101]

meses_unicos = juros_consignado_privado['data'].dt.to_period('M').unique()
lista_resultados = []

for mes in meses_unicos:
    df_mes = juros_consignado_privado[juros_consignado_privado['data'].dt.to_period('M') == mes]
    ultima_data_do_mes = df_mes['data'].max()
    df_filtrado = df_mes[df_mes['data'] == ultima_data_do_mes]
    lista_resultados.append(df_filtrado)

juros_consignado_privado = pd.concat(lista_resultados)
juros_consignado_privado['data'] = juros_consignado_privado['data'].dt.strftime('%Y-%m')
#juros = juros.drop(columns=['data'])

juros_consignado_privado.to_sql(
    'juros_consignado_privado',
    engine,
    if_exists='replace',
    index=False,
    chunksize=1000
)


7765

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('sqlite:///dados/banking.db')

juros = pd.read_csv('dados/Taxa de juros de modalidades diárias e mensais.csv', sep=',', encoding='latin1', on_bad_lines='warn')
juros = juros.rename(columns={'cnpj8': 'cnpj', 'FimPeriodo': 'data'})
juros['data'] = pd.to_datetime(juros['data'])
juros = juros.drop(columns=['InicioPeriodo', 'TaxaJurosAoMes'])
#juros = juros.set_index('data')

meses_unicos = juros['data'].dt.to_period('M').unique()
lista_resultados = []

for mes in meses_unicos:
    df_mes = juros[juros['data'].dt.to_period('M') == mes]
    ultima_data_do_mes = df_mes['data'].max()
    df_filtrado = df_mes[df_mes['data'] == ultima_data_do_mes]
    lista_resultados.append(df_filtrado)

juros_mensal = pd.concat(lista_resultados)
juros_mensal['data'] = juros_mensal['data'].dt.strftime('%Y-%m')
#juros = juros.drop(columns=['data'])

juros_mensal.to_sql(
    'juros_mensal',
    engine,
    if_exists='replace',
    index=False,
    chunksize=1000
)

In [6]:
import pandas as pd
#from sqlalchemy import create_engine

#engine = create_engine('sqlite:///dados/banking.db')

cdi = pd.read_csv('dados/CDI.xls', sep='\\t', encoding='latin1', engine= 'python')
cdi = cdi.rename(columns={'Data': 'data', 'Média': 'cdi', 'Taxa SELIC': 'selic'})
cdi['data'] = pd.to_datetime(cdi['data'], format='mixed')
cdi = cdi.sort_values(by='data')
cdi_mensal = cdi.groupby(cdi['data'].dt.to_period('M')).last()
cdi_mensal.index = pd.to_datetime(cdi_mensal.index.to_timestamp())
cdi_mensal.index = cdi_mensal['data'].dt.strftime('%Y-%m')
cdi_mensal = cdi_mensal.drop(columns= ['data'])

cdi_mensal.to_sql(
    'cdi_mensal',
    engine,
    if_exists='replace',
    index=True,
    chunksize=1000
)


192

In [11]:
import pandas as pd
import os

#Dados balanços

def juntar_csvs(caminho_da_pasta, valor_documento, nome_documento='DOCUMENTO'):
    todos_dataframes_filtrados = []

    # Verificar se o caminho da pasta existe
    if not os.path.isdir(caminho_da_pasta):
        print(f"Erro: O caminho da pasta '{caminho_da_pasta}' não existe.")
        return pd.DataFrame()

    # Listar todos os arquivos na pasta
    arquivos_na_pasta = os.listdir(caminho_da_pasta)

    # Filtrar apenas arquivos CSV
    arquivos_csv = [f for f in arquivos_na_pasta if f.lower().endswith('.csv')]

    if not arquivos_csv:
        print(f"Nenhum arquivo CSV encontrado na pasta: '{caminho_da_pasta}'")
        return pd.DataFrame()

    print(f"Arquivos CSV encontrados: {arquivos_csv}")

    for arquivo_csv in arquivos_csv:
        caminho_completo_arquivo = os.path.join(caminho_da_pasta, arquivo_csv)
        print(f"Processando arquivo: {arquivo_csv}")

        try:
            # Tenta ler o arquivo CSV
            df_temp = pd.read_csv(caminho_completo_arquivo, sep=';', encoding='latin1', skiprows=3, decimal=',', on_bad_lines='warn')

            # Verificar se a coluna de filtro existe no DataFrame
            if nome_documento in df_temp.columns:
                df_temp[nome_documento] = pd.to_numeric(df_temp[nome_documento], errors='coerce')

                df_filtrado = df_temp[
                    (df_temp[nome_documento] == valor_documento)
                ].copy()

                if not df_filtrado.empty:
                    todos_dataframes_filtrados.append(df_filtrado)
                    print(f"  - {len(df_filtrado)} linhas adicionadas do arquivo {arquivo_csv}.")
                
                else:
                    print(f"  - Nenhuma linha corresponde aos filtros em {arquivo_csv}.")

            else:
                missing_cols = []
                if nome_documento not in df_temp.columns:
                    missing_cols.append(nome_documento)
                print(f"  - Coluna(s) {', '.join(missing_cols)} não encontrada(s) no arquivo {arquivo_csv}. Pulando este arquivo.")

        except pd.errors.EmptyDataError:
            print(f"Erro: O arquivo '{arquivo_csv}' está vazio. Ignorando.")
        except Exception as e:
            print(f"Erro ao ler ou processar '{arquivo_csv}': {e}")

    if todos_dataframes_filtrados:
        # Concatenar todos os DataFrames filtrados
        df_final = pd.concat(todos_dataframes_filtrados, ignore_index=True)
        print("\nProcessamento concluído. DataFrame final criado.")
        return df_final
    else:
        print("\nNenhum dado correspondente ao filtro foi encontrado em nenhum arquivo.")
        return pd.DataFrame()


Arquivos CSV encontrados: ['201209BANCOS.CSV', '202503BANCOS.CSV', '202103BANCOS.CSV', '202005BANCOS.CSV', '202101BANCOS.CSV', '202111BANCOS.CSV', '202402BANCOS.CSV', '201906BANCOS.CSV', '202112BANCOS.CSV', '202406BANCOS.CSV', '201303BANCOS.CSV', '201903BANCOS.CSV', '201707BANCOS.CSV', '201811BANCOS.CSV', '201002BANCOS.CSV', '202308BANCOS.CSV', '201306BANCOS.CSV', '201309BANCOS.CSV', '201307BANCOS.CSV', '201202BANCOS.CSV', '202207BANCOS.CSV', '201508BANCOS.CSV', '201003BANCOS.CSV', '201201BANCOS.CSV', '202410BANCOS.CSV', '201609BANCOS.CSV', '201607BANCOS.CSV', '201910BANCOS.CSV', '201905BANCOS.CSV', '201501BANCOS.CSV', '201109BANCOS.CSV', '202110BANCOS.CSV', '201809BANCOS.CSV', '201412BANCOS.CSV', '201602BANCOS.CSV', '201709BANCOS.CSV', '201502BANCOS.CSV', '202106BANCOS.CSV', '201603BANCOS.CSV', '201708BANCOS.CSV', '201101BANCOS.CSV', '202311BANCOS.CSV', '201512BANCOS.CSV', '201504BANCOS.CSV', '202404BANCOS.CSV', '201704BANCOS.CSV', '201511BANCOS.CSV', '201205BANCOS.CSV', '201102BANCOS

In [ ]:


bp_bancos = juntar_csvs('dados/dados_bancos', valor_documento=4016)
bp_sociedades = juntar_csvs('dados/dados_sociedades', valor_documento=4016)
bp_cooperativas = juntar_csvs('dados/dados_cooperativas', valor_documento=4016)


In [51]:
bp_bancos = bp_bancos.rename(columns={'CNPJ': 'cnpj', '#DATA_BASE': 'data'})
bp_bancos['cnpj'] = bp_bancos['cnpj'].astype(str)
bp_bancos['data'] = bp_bancos['data'].astype(str)
bp_bancos['data'] = pd.to_datetime(bp_bancos['data'], format='%Y%m').dt.strftime('%Y-%m')


bp_bancos.to_sql(
    'bp_bancos',
    engine,
    if_exists='replace',
    index=False,
    chunksize=1000
)


bp_sociedades = bp_sociedades.rename(columns={'CNPJ': 'cnpj', '#DATA_BASE': 'data'})
bp_sociedades['data'] = bp_sociedades['data'].astype(str)
bp_sociedades['data'] = pd.to_datetime(bp_sociedades['data'], format='%Y%m').dt.strftime('%Y-%m')
bp_sociedades.to_sql(
    'bp_sociedades',
    engine,
    if_exists='replace',
    index=False,
    chunksize=1000
)


bp_cooperativas = bp_cooperativas.rename(columns={'CNPJ': 'cnpj','#DATA_BASE': 'data'})
bp_cooperativas['cnpj'] = bp_cooperativas['cnpj'].astype(str)
bp_cooperativas['data'] = bp_cooperativas['data'].astype(str)
bp_cooperativas['data'] = pd.to_datetime(bp_cooperativas['data'], format='%Y%m').dt.strftime('%Y-%m')
bp_cooperativas.to_sql(
    'bp_cooperativas',
    engine,
    if_exists='replace',
    index=False,
    chunksize=1000  
)

1821336

In [34]:
balancetes_bancos = juntar_csvs('dados/dados_bancos', valor_documento=4010)
balancetes_sociedades = juntar_csvs('dados/dados_sociedades', valor_documento=4010)
balancetes_cooperativas = juntar_csvs('dados/dados_cooperativas', valor_documento=4010)


Arquivos CSV encontrados: ['201209BANCOS.CSV', '202503BANCOS.CSV', '202103BANCOS.CSV', '202005BANCOS.CSV', '202101BANCOS.CSV', '202111BANCOS.CSV', '202402BANCOS.CSV', '201906BANCOS.CSV', '202112BANCOS.CSV', '202406BANCOS.CSV', '201303BANCOS.CSV', '201903BANCOS.CSV', '201707BANCOS.CSV', '201811BANCOS.CSV', '201002BANCOS.CSV', '202308BANCOS.CSV', '201306BANCOS.CSV', '201309BANCOS.CSV', '201307BANCOS.CSV', '201202BANCOS.CSV', '202207BANCOS.CSV', '201508BANCOS.CSV', '201003BANCOS.CSV', '201201BANCOS.CSV', '202410BANCOS.CSV', '201609BANCOS.CSV', '201607BANCOS.CSV', '201910BANCOS.CSV', '201905BANCOS.CSV', '201501BANCOS.CSV', '201109BANCOS.CSV', '202110BANCOS.CSV', '201809BANCOS.CSV', '201412BANCOS.CSV', '201602BANCOS.CSV', '201709BANCOS.CSV', '201502BANCOS.CSV', '202106BANCOS.CSV', '201603BANCOS.CSV', '201708BANCOS.CSV', '201101BANCOS.CSV', '202311BANCOS.CSV', '201512BANCOS.CSV', '201504BANCOS.CSV', '202404BANCOS.CSV', '201704BANCOS.CSV', '201511BANCOS.CSV', '201205BANCOS.CSV', '201102BANCOS

In [35]:
balancetes_bancos['#DATA_BASE'] = balancetes_bancos['#DATA_BASE'].fillna(balancetes_bancos['DATA'])
balancetes_bancos['NOME_INSTITUICAO'] = balancetes_bancos['NOME_INSTITUICAO'].fillna(balancetes_bancos['NOME_INSTITUICAO'])
balancetes_bancos['NOME_CONTA'] = balancetes_bancos['NOME_CONTA'].fillna(balancetes_bancos['NOME CONTA'])
balancetes_bancos = balancetes_bancos.rename(columns={'SALDO': 'saldo'})
balancetes_bancos.columns = balancetes_bancos.columns.str.strip()
balancetes_bancos['SALDO'] = balancetes_bancos['SALDO'].fillna(balancetes_bancos['saldo'])
balancetes_bancos = balancetes_bancos.drop(columns=['DATA', 'NOME CONTA', 'NOME INSTITUICAO', 'saldo'])

balancetes_bancos = balancetes_bancos.rename(columns={'CNPJ': 'cnpj', '#DATA_BASE': 'data'})
balancetes_bancos['cnpj'] = balancetes_bancos['cnpj'].astype(str)
balancetes_bancos['data'] = balancetes_bancos['data'].astype(float)
balancetes_bancos['data'] = pd.to_datetime(balancetes_bancos['data'], format='%Y%m').dt.strftime('%Y-%m')


In [36]:

balancetes_bancos.to_sql(
    'balancetes_bancos',
    engine,
    if_exists='replace',
    index=False,
    chunksize=1000
)


3164840

In [39]:
balancetes_sociedades['#DATA_BASE'] = balancetes_sociedades['#DATA_BASE'].fillna(balancetes_sociedades['DATA'])
balancetes_sociedades['NOME_INSTITUICAO'] = balancetes_sociedades['NOME_INSTITUICAO'].fillna(balancetes_sociedades['NOME_INSTITUICAO'])
balancetes_sociedades['NOME_CONTA'] = balancetes_sociedades['NOME_CONTA'].fillna(balancetes_sociedades['NOME CONTA'])
balancetes_sociedades = balancetes_sociedades.rename(columns={'SALDO': 'saldo'})
balancetes_sociedades.columns = balancetes_sociedades.columns.str.strip()
balancetes_sociedades['SALDO'] = balancetes_sociedades['SALDO'].fillna(balancetes_sociedades['saldo'])
balancetes_sociedades = balancetes_sociedades.drop(columns=['DATA', 'NOME CONTA', 'NOME INSTITUICAO', 'saldo'])

balancetes_sociedades = balancetes_sociedades.rename(columns={'CNPJ': 'cnpj', '#DATA_BASE': 'data'})
balancetes_sociedades['data'] = pd.to_datetime(balancetes_sociedades['data'], format='%Y%m').dt.strftime('%Y-%m')
balancetes_sociedades.to_sql(
    'balancetes_sociedades',
    engine,
    if_exists='replace',
    index=False,
    chunksize=1000
)

3544620

In [ ]:
balancetes_cooperativas['#DATA_BASE'] = balancetes_cooperativas['#DATA_BASE'].fillna(balancetes_cooperativas['DATA'])
balancetes_cooperativas['NOME_INSTITUICAO'] = balancetes_cooperativas['NOME_INSTITUICAO'].fillna(balancetes_cooperativas['NOME_INSTITUICAO'])
balancetes_cooperativas['NOME_CONTA'] = balancetes_cooperativas['NOME_CONTA'].fillna(balancetes_cooperativas['NOME_CONTA'])
balancetes_cooperativas = balancetes_cooperativas.rename(columns={'SALDO': 'saldo'})
balancetes_cooperativas.columns = balancetes_cooperativas.columns.str.strip()
balancetes_cooperativas['SALDO'] = balancetes_cooperativas['SALDO'].fillna(balancetes_cooperativas['saldo'])
balancetes_cooperativas = balancetes_cooperativas.drop(columns=['DATA', 'NOME CONTA', 'NOME INSTITUICAO', 'saldo'])

balancetes_cooperativas = balancetes_cooperativas.rename(columns={'CNPJ': 'cnpj','#DATA_BASE': 'data'})
balancetes_cooperativas['cnpj'] = balancetes_cooperativas['cnpj'].astype(str)
#balancetes_cooperativas['data'] = balancetes_cooperativas['data'].astype(str)
balancetes_cooperativas['data'] = pd.to_datetime(balancetes_cooperativas['data'], format='%Y%m').dt.strftime('%Y-%m')
balancetes_cooperativas.to_sql(
    'balancetes_cooperativas',
    engine,
    if_exists='replace',
    index=False,
    chunksize=1000  
)


In [42]:
balancetes = pd.concat([balancetes_bancos, balancetes_sociedades, balancetes_cooperativas], ignore_index=True)
balanceteas = balancetes

balancetes.to_sql(
    'balancetes',
    engine,
    if_exists='replace',
    index=False,
    chunksize=1000
)


17695885

In [66]:
balancos = pd.concat([bp_bancos, bp_sociedades, bp_cooperativas], ignore_index=True)

balancos.to_sql(
    'balancos',
    engine,
    if_exists='replace',
    index=False,
    chunksize=1000
)

2745643

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('dados/banking.db')

query = """
WITH RankedData AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY data ORDER BY "SALDO" DESC) AS rn
    FROM
        balancos
    WHERE
        "CONTA" = '16000001'
)
SELECT
    *
FROM
    RankedData
WHERE
    rn <= 5;
"""

top5_credito = pd.read_sql_query(query, conn)

query = """
SELECT
    *
FROM
    balancos
WHERE
    "CONTA" = '16000001'
"""

operações_de_credito = pd.read_sql_query(query, conn)

conn.close()

saldo_top5 = pd.DataFrame(top5_credito.groupby('data')['SALDO'].sum())
saldo_top5 = saldo_top5.rename(columns={'SALDO': 'top5'})

saldo_total = pd.DataFrame(operações_de_credito.groupby('data')['SALDO'].sum())
saldo_total = saldo_total.rename(columns={'SALDO': 'total'})
saldo_total = saldo_total.join(saldo_top5['top5'])

saldo_total['mktshare'] = saldo_total['top5'] / saldo_total['total']


In [8]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('dados/banking.db')

query = """
WITH RankedData AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY data ORDER BY "SALDO" DESC) AS rn
    FROM
        balancos
    WHERE
        "CONTA" = '41000007'
)
SELECT
    *
FROM
    RankedData
WHERE
    rn <= 5;
"""

top5_depositos = pd.read_sql_query(query, conn)

query = """
SELECT
    *
FROM
    balancos
WHERE
    "CONTA" = '41000007'
"""

depositos_totais = pd.read_sql_query(query, conn)

conn.close()

dep_saldo_top5 = pd.DataFrame(top5_depositos.groupby('data')['SALDO'].sum())
dep_saldo_top5 = dep_saldo_top5.rename(columns={'SALDO': 'top5'})

dep_saldo_total = pd.DataFrame(depositos_totais.groupby('data')['SALDO'].sum())
dep_saldo_total = dep_saldo_total.rename(columns={'SALDO': 'total'})
dep_saldo_total = dep_saldo_total.join(dep_saldo_top5['top5'])

dep_saldo_total['mktshare'] = dep_saldo_total['top5'] / dep_saldo_total['total']


In [26]:
conn = sqlite3.connect('dados/banking.db')

query = """
WITH RankedData AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY data ORDER BY "SALDO" DESC) AS rn
    FROM
        balancos
    WHERE
        "CONTA" = '39999993'
)
SELECT
    *
FROM
    RankedData
WHERE
    rn <= 5;
"""

top5_ativos = pd.read_sql_query(query, conn)

query = """
SELECT
    *
FROM
    balancos
WHERE
    "CONTA" = '39999993'
"""

ativos_totais = pd.read_sql_query(query, conn)

conn.close()

ativo_saldo_top5 = pd.DataFrame(top5_ativos.groupby('data')['SALDO'].sum())
ativo_saldo_top5 = ativo_saldo_top5.rename(columns={'SALDO': 'top5'})

ativo_saldo_total = pd.DataFrame(ativos_totais.groupby('data')['SALDO'].sum())
ativo_saldo_total = ativo_saldo_total.rename(columns={'SALDO': 'total'})
ativo_saldo_total = ativo_saldo_total.join(ativo_saldo_top5['top5'])

ativo_saldo_total['mktshare'] = ativo_saldo_total['top5'] / ativo_saldo_total['total']
